# Analysis Q-Wave Algorithms

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from fau_colors import cmaps, register_fausans_font

from pepbench.data_handling import compute_pep_performance_metrics
from pepbench.datasets import EmpkinsDataset
from pepbench.io import load_challenge_results_from_folder
from pepbench.plotting.results import (
    boxplot_algorithm_performance,
    regplot_error_heart_rate,
    regplot_pep_heart_rate,
    residual_plot_pep,
    residual_plot_pep_heart_rate,
    residual_plot_pep_phase,
    residual_plot_pep_subject,
    violinplot_algorithm_performance,
)

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties_light)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

In [ ]:
root_path = Path("../../")

In [ ]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(empkins_base_path)

In [ ]:
result_path = root_path.joinpath("results")

In [ ]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=True, only_labeled=True)
dataset_empkins

In [ ]:
algo_levels = ["q_wave_algorithm", "b_point_algorithm", "outlier_correction_algorithm"]
algo_level_mapping = dict(zip(algo_levels, ["Q-Wave Algorithm", "B-Point Algorithm", "Outlier Correction"]))

In [ ]:
results_empkins = load_challenge_results_from_folder(
    result_path.joinpath("empkins_dataset_reference_b_point"),
    index_cols_per_sample=["participant", "condition", "phase"],
)

In [ ]:
results_guardian = load_challenge_results_from_folder(
    result_path.joinpath("guardian_dataset_reference_b_point"), index_cols_per_sample=["participant", "phase"]
)

In [ ]:
metrics_empkins = compute_pep_performance_metrics(results_empkins.per_sample, num_heartbeats=results_empkins.agg_total)
metrics_empkins

In [ ]:
fig, ax = boxplot_algorithm_performance(
    results_empkins.per_sample,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    palette=cmaps.faculties_light,
    figsize=(12, 5),
)

In [ ]:
fig, ax = violinplot_algorithm_performance(
    results_empkins.per_sample,
    metric="error_per_sample_ms",
    showmeans=True,
    palette=cmaps.faculties_light,
    figsize=(12, 5),
)

In [ ]:
fig, ax = violinplot_algorithm_performance(
    results_empkins.per_sample,
    metric="absolute_error_per_sample_ms",
    showmeans=True,
    palette=cmaps.faculties_light,
    figsize=(12, 5),
)

In [ ]:
algo_combination = ("dwt-neurokit", "b-point-reference", "none")
algo_combination_2 = ("r-peak-diff-32-ms", "b-point-reference", "none")

In [ ]:
fig, ax = residual_plot_pep(results_empkins.per_sample, algo_combination)
fig, ax = residual_plot_pep(results_empkins.per_sample, algo_combination_2)

In [ ]:
fig, ax = residual_plot_pep_subject(results_empkins.per_sample, algo_combination, alpha=0.5)
fig, ax = residual_plot_pep_subject(results_empkins.per_sample, algo_combination_2, alpha=0.5)

In [ ]:
fig, ax = residual_plot_pep_phase(results_empkins.per_sample, algo_combination)
fig, ax = residual_plot_pep_phase(results_empkins.per_sample, algo_combination_2)

In [ ]:
fig, ax = residual_plot_pep_heart_rate(results_empkins.per_sample, algo_combination, alpha=0.5)
fig, ax = residual_plot_pep_heart_rate(results_empkins.per_sample, algo_combination_2, alpha=0.5)

In [ ]:
fig, ax = regplot_error_heart_rate(
    results_empkins.per_sample, algo_combination, error_metric="absolute_error_per_sample_ms"
)
fig, ax = regplot_error_heart_rate(
    results_empkins.per_sample, algo_combination_2, error_metric="absolute_error_per_sample_ms"
)

In [ ]:
fig, ax = regplot_pep_heart_rate(results_empkins.per_sample, algo_combination)

In [ ]:
# for key, df in results_empkins.per_sample.groupby(algo_levels):
#    fig, ax = plot_error_residual(df, algorithm=key)

In [ ]:
results_empkins.per_sample[("nan_reason", "estimated")].groupby(algo_levels).value_counts().to_frame()

In [ ]:
algo_test = results_empkins.per_sample.xs(("dwt-neurokit", "b-point-reference", "none"))
isna_mask = algo_test[("pep_ms", "estimated")].isna()

algo_test.loc[isna_mask]